# 🌐 Modbus TCP Externo + Captura de Tramas con Scapy

**Tutorial completo** para conectarse externamente a un servidor Modbus TCP, leer variables y capturar/analizar las tramas de red.

## 🎯 Objetivos

1. **Entender Modbus TCP**: Protocolo industrial basado en TCP/IP
2. **Estructura de registros**: Input Registers vs Holding Registers
3. **Conexión externa**: Conectarse desde fuera del servidor Docker
4. **Análisis de tramas**: Capturar y decodificar paquetes con Scapy
5. **Visualización**: Interpretar la estructura de mensajes Modbus

---

## 📖 ¿Qué es Modbus TCP?

**Modbus** es el protocolo de comunicación industrial más usado en el mundo.

### Historia
- Creado por Modicon (ahora Schneider Electric) en 1979
- Originalmente para comunicación serie (Modbus RTU)
- En 1999 se adaptó a TCP/IP → **Modbus TCP**

### Ventajas
- ✅ **Simple**: Fácil de implementar
- ✅ **Abierto**: Especificación pública
- ✅ **Universal**: Soportado por miles de dispositivos
- ✅ **Eficiente**: Overhead mínimo de protocolo

### Arquitectura Cliente-Servidor
```
Cliente (HMI/SCADA)  ←→  Servidor (PLC/Sensor)
     Solicita datos          Responde con datos
     Puerto cualquiera       Puerto 502 (estándar)
```

## 📊 Tipos de Registros en Modbus

Modbus define **4 tipos de datos** organizados en "tablas":

| Tipo de Registro | Acceso | Tamaño | Rango Direcciones | Función Lectura | Función Escritura | Uso Típico |
|------------------|--------|--------|-------------------|-----------------|-------------------|------------|
| **Coils** | R/W | 1 bit | 00001 - 09999 | 0x01 | 0x05, 0x0F | Salidas digitales (relés, LEDs) |
| **Discrete Inputs** | R | 1 bit | 10001 - 19999 | 0x02 | - | Entradas digitales (sensores ON/OFF) |
| **Input Registers** | R | 16 bits | 30001 - 39999 | 0x04 | - | Sensores analógicos (temperatura, presión) |
| **Holding Registers** | R/W | 16 bits | 40001 - 49999 | 0x03 | 0x06, 0x10 | Parámetros de configuración |

### 🔍 Diferencias clave

#### Input Registers (0x04) - Solo Lectura
- 📡 **Sensores**: Valores que cambian constantemente
- 🚫 **No modificables**: El cliente solo puede leerlos
- 📊 **Ejemplo**: Temperatura actual, nivel de líquido, velocidad motor

#### Holding Registers (0x03) - Lectura/Escritura
- ⚙️ **Parámetros**: Valores que pueden modificarse
- ✏️ **Configurables**: El cliente puede escribirlos
- 🎚️ **Ejemplo**: Setpoint de temperatura, límite de velocidad, ganancias PID

### 📐 Direccionamiento

**Importante**: Modbus usa direcciones **base 0** internamente, pero las especificaciones usan **base 1**.

```
Holding Register 40001 → Dirección interna: 0
Holding Register 40002 → Dirección interna: 1
Input Register 30001 → Dirección interna: 0
```

En **PyModbus** siempre usamos **direcciones base 0**.

## 📦 Estructura de Trama Modbus TCP

Cada mensaje Modbus TCP tiene dos partes:

### 1️⃣ MBAP Header (Modbus Application Protocol Header)
```
+----------------+----------------+
| Transaction ID |  Protocol ID   |  ← 4 bytes
+----------------+----------------+
|     Length     |   Unit ID      |  ← 4 bytes
+----------------+----------------+
         ↓              ↓
    Correlaciona    Siempre 0
    petición/       (TCP/IP)
    respuesta
```

| Campo | Tamaño | Descripción |
|-------|--------|-------------|
| **Transaction ID** | 2 bytes | Identificador único para correlacionar peticiones/respuestas |
| **Protocol ID** | 2 bytes | Siempre `0x0000` (identifica Modbus) |
| **Length** | 2 bytes | Número de bytes siguientes (Unit ID + PDU) |
| **Unit ID** | 1 byte | Identificador del esclavo (normalmente `0x01` o `0xFF`) |

### 2️⃣ PDU (Protocol Data Unit)
```
+------------------+------------------+
| Function Code    |      Data        |
+------------------+------------------+
   1 byte             N bytes
```

| Campo | Descripción | Ejemplo |
|-------|-------------|---------|
| **Function Code** | Tipo de operación (0x03, 0x04, 0x06, etc.) | `0x03` = Read Holding Registers |
| **Data** | Direcciones, valores, cantidad de registros | `0x0000 0x0008` = Desde addr 0, leer 8 registros |

### 📝 Ejemplo: Lectura de 8 Holding Registers

**Petición del Cliente:**
```
MBAP Header:
  00 01          ← Transaction ID
  00 00          ← Protocol ID (Modbus)
  00 06          ← Length (6 bytes siguientes)
  01             ← Unit ID

PDU:
  03             ← Function Code (Read Holding Registers)
  00 00          ← Dirección inicio (0)
  00 08          ← Cantidad de registros (8)
```

**Respuesta del Servidor:**
```
MBAP Header:
  00 01          ← Transaction ID (el mismo)
  00 00          ← Protocol ID
  00 13          ← Length (19 bytes siguientes)
  01             ← Unit ID

PDU:
  03             ← Function Code
  10             ← Byte Count (16 bytes = 8 registros × 2 bytes)
  00 64          ← Registro 0: 100
  00 C8          ← Registro 1: 200
  ...            ← Registros 2-7
```

---

## 🔧 Paso 1: Instalación de Dependencias

Necesitamos dos bibliotecas principales:

In [ ]:
!pip install -q pymodbus==3.6.4 scapy
import pymodbus
import scapy
print(f"✅ PyModbus versión: {pymodbus.__version__}")
print(f"✅ Scapy versión: {scapy.__version__}")

### 📚 Bibliotecas instaladas

- **PyModbus**: Cliente Modbus TCP para comunicación con el servidor
- **Scapy**: Herramienta avanzada de manipulación de paquetes de red

### ¿Por qué Scapy?

A diferencia de PyShark (usado en el otro notebook), **Scapy** permite:

- ✅ **Crear paquetes personalizados** desde cero
- ✅ **Manipular campos** específicos de protocolos
- ✅ **Enviar/Recibir** paquetes manualmente
- ✅ **Análisis en tiempo real** sin necesidad de archivos PCAP

---

## 🔧 Paso 2: Importar Módulos

In [ ]:
from pymodbus.client import ModbusTcpClient
from scapy.all import sniff, TCP, IP, Raw, conf
import time

# Configurar Scapy para no mostrar advertencias
conf.verb = 0

---

## 🌐 Paso 3: Configuración de Conexión Externa

### Diferencia: Conexión Interna vs Externa

| Aspecto | Conexión Interna (Docker) | Conexión Externa |
|---------|---------------------------|------------------|
| **IP** | `172.25.0.10` (red Docker) | `65.109.226.13` (IP pública) |
| **Puerto** | `502` (interno) | `5020` (mapeado externamente) |
| **Uso** | Entre contenedores | Desde cualquier cliente en Internet |
| **Firewall** | No necesario | Debe estar abierto |

### 📝 Variables de nuestro servidor

**Input Registers** (Solo Lectura - Sensores):
- `30001` (addr 0): Temperatura (°C)
- `30002` (addr 1): Presión (bar)
- `30003` (addr 2): Nivel tanque (%)
- `30004` (addr 3): Velocidad motor (RPM)

**Holding Registers** (Lectura/Escritura - Parámetros):
- `40001` (addr 0): Setpoint temperatura (°C)
- `40002` (addr 1): Límite presión (bar)
- `40003` (addr 2): Límite nivel (%)
- `40004` (addr 3): Límite velocidad (RPM)

### 🔗 Conectarse al servidor

In [ ]:
# Configuración del servidor (AJUSTA ESTOS VALORES)
SERVER_IP = "65.109.226.13"  # IP pública del servidor
SERVER_PORT = 5020            # Puerto externo mapeado

# Crear cliente Modbus TCP
client = ModbusTcpClient(host=SERVER_IP, port=SERVER_PORT, timeout=3)

# Intentar conectar
if client.connect():
    print(f"✅ Conectado a {SERVER_IP}:{SERVER_PORT}")
else:
    print(f"❌ Error: No se pudo conectar a {SERVER_IP}:{SERVER_PORT}")
    print("   Verifica que el servidor esté corriendo: ./start.sh")

---

## 📖 Paso 4: Leer Input Registers (Sensores)

In [ ]:
# Leer Input Registers (función 0x04)
response = client.read_input_registers(address=0, count=4, slave=1)

if not response.isError():
    print("📊 Input Registers (Sensores - Solo Lectura):")
    print(f"  30001 - Temperatura: {response.registers[0]} °C")
    print(f"  30002 - Presión: {response.registers[1]} bar")
    print(f"  30003 - Nivel tanque: {response.registers[2]} %")
    print(f"  30004 - Velocidad motor: {response.registers[3]} RPM")
else:
    print(f"❌ Error al leer Input Registers: {response}")

---

## 📖 Paso 5: Leer Holding Registers (Parámetros)

In [ ]:
# Leer Holding Registers (función 0x03)
response = client.read_holding_registers(address=0, count=4, slave=1)

if not response.isError():
    print("⚙️ Holding Registers (Parámetros - Lectura/Escritura):")
    print(f"  40001 - Setpoint Temperatura: {response.registers[0]} °C")
    print(f"  40002 - Límite Presión: {response.registers[1]} bar")
    print(f"  40003 - Límite Nivel: {response.registers[2]} %")
    print(f"  40004 - Límite Velocidad: {response.registers[3]} RPM")
else:
    print(f"❌ Error al leer Holding Registers: {response}")

---

## ✏️ Paso 6: Escribir un Holding Register

Modificaremos el **Setpoint de Temperatura** (registro 40001).

In [ ]:
# Escribir un registro (función 0x06)
nuevo_setpoint = 250

response = client.write_register(address=0, value=nuevo_setpoint, slave=1)

if not response.isError():
    print(f"✅ Setpoint modificado a {nuevo_setpoint} °C")
    
    # Verificar el cambio
    verify = client.read_holding_registers(address=0, count=1, slave=1)
    if not verify.isError():
        print(f"✅ Verificado: Setpoint = {verify.registers[0]} °C")
else:
    print(f"❌ Error al escribir: {response}")

---

## 📡 Paso 7: Capturar Tráfico Modbus con Scapy

Ahora vamos a **capturar las tramas** que genera nuestra comunicación Modbus.

### 🔍 ¿Qué haremos?

1. **Iniciar captura** de paquetes en segundo plano
2. **Realizar operaciones** Modbus (leer/escribir)
3. **Detener captura** y analizar los paquetes

### ⚙️ Función de captura

In [ ]:
from threading import Thread, Event
import queue

# Variables globales para la captura
captured_packets = queue.Queue()
stop_sniffing = Event()

def packet_handler(pkt):
    """Manejador que almacena paquetes Modbus"""
    if TCP in pkt and (pkt[TCP].sport == SERVER_PORT or pkt[TCP].dport == SERVER_PORT):
        if Raw in pkt:
            captured_packets.put(pkt)

def start_capture():
    """Inicia captura en segundo plano"""
    def capture_thread():
        sniff(
            filter=f"tcp port {SERVER_PORT}",
            prn=packet_handler,
            stop_filter=lambda x: stop_sniffing.is_set(),
            store=False
        )
    
    thread = Thread(target=capture_thread, daemon=True)
    thread.start()
    print("🎯 Captura iniciada...")

def stop_capture():
    """Detiene la captura"""
    stop_sniffing.set()
    time.sleep(0.5)
    print(f"⏹️ Captura detenida. Total de paquetes: {captured_packets.qsize()}")

---

## 📊 Paso 8: Ejecutar Operaciones y Capturar Tramas

In [ ]:
# Limpiar captura anterior
while not captured_packets.empty():
    captured_packets.get()
stop_sniffing.clear()

# Iniciar captura
start_capture()
time.sleep(1)  # Esperar que inicie

# Realizar operaciones Modbus
print("\n📖 Leyendo Input Registers...")
client.read_input_registers(address=0, count=4, slave=1)

time.sleep(0.5)

print("📖 Leyendo Holding Registers...")
client.read_holding_registers(address=0, count=4, slave=1)

time.sleep(0.5)

print("✏️ Escribiendo Holding Register...")
client.write_register(address=0, value=300, slave=1)

time.sleep(1)

# Detener captura
stop_capture()

---

## 🔍 Paso 9: Analizar las Tramas Capturadas

Vamos a **decodificar** los paquetes capturados y mostrar la estructura Modbus.

In [ ]:
import struct

def decode_modbus_packet(pkt):
    """Decodifica un paquete Modbus TCP"""
    if Raw not in pkt:
        return None
    
    payload = bytes(pkt[Raw].load)
    
    if len(payload) < 8:  # Mínimo: MBAP (7) + Function Code (1)
        return None
    
    # Decodificar MBAP Header
    transaction_id = struct.unpack('>H', payload[0:2])[0]
    protocol_id = struct.unpack('>H', payload[2:4])[0]
    length = struct.unpack('>H', payload[4:6])[0]
    unit_id = payload[6]
    
    # Decodificar PDU
    function_code = payload[7]
    
    # Determinar dirección
    direction = "Cliente → Servidor" if pkt[TCP].dport == SERVER_PORT else "Servidor → Cliente"
    
    # Diccionario de funciones
    functions = {
        0x01: "Read Coils",
        0x02: "Read Discrete Inputs",
        0x03: "Read Holding Registers",
        0x04: "Read Input Registers",
        0x05: "Write Single Coil",
        0x06: "Write Single Register",
        0x0F: "Write Multiple Coils",
        0x10: "Write Multiple Registers"
    }
    
    return {
        'direction': direction,
        'transaction_id': transaction_id,
        'protocol_id': protocol_id,
        'length': length,
        'unit_id': unit_id,
        'function_code': function_code,
        'function_name': functions.get(function_code, f"Unknown (0x{function_code:02X})"),
        'raw_data': payload.hex()
    }

In [ ]:
# Analizar todos los paquetes capturados
packets_list = []
while not captured_packets.empty():
    packets_list.append(captured_packets.get())

print(f"📦 Total de paquetes capturados: {len(packets_list)}\n")

for i, pkt in enumerate(packets_list, 1):
    decoded = decode_modbus_packet(pkt)
    if decoded:
        print(f"{'='*70}")
        print(f"Paquete #{i} - {decoded['direction']}")
        print(f"{'='*70}")
        print(f"  MBAP Header:")
        print(f"    Transaction ID: 0x{decoded['transaction_id']:04X}")
        print(f"    Protocol ID:    0x{decoded['protocol_id']:04X} (Modbus)")
        print(f"    Length:         {decoded['length']} bytes")
        print(f"    Unit ID:        {decoded['unit_id']}")
        print(f"  PDU:")
        print(f"    Function Code:  0x{decoded['function_code']:02X} - {decoded['function_name']}")
        print(f"  Raw Data (hex):   {decoded['raw_data']}")
        print()

---

## 🔬 Paso 10: Análisis Detallado de una Trama

Vamos a **diseccionar** una trama completa byte por byte.

In [ ]:
def analyze_modbus_frame(payload_hex):
    """Análisis detallado byte por byte de una trama Modbus"""
    payload = bytes.fromhex(payload_hex)
    
    print("🔬 ANÁLISIS DETALLADO DE TRAMA MODBUS TCP")
    print("="*70)
    print(f"Trama completa (hex): {payload_hex}")
    print(f"Longitud total: {len(payload)} bytes\n")
    
    print("📋 MBAP HEADER (7 bytes)")
    print("-"*70)
    
    # Transaction ID
    trans_id = struct.unpack('>H', payload[0:2])[0]
    print(f"  Bytes 0-1:  {payload[0:2].hex().upper()}  → Transaction ID = {trans_id}")
    
    # Protocol ID
    proto_id = struct.unpack('>H', payload[2:4])[0]
    print(f"  Bytes 2-3:  {payload[2:4].hex().upper()}  → Protocol ID = {proto_id} (Modbus)")
    
    # Length
    length = struct.unpack('>H', payload[4:6])[0]
    print(f"  Bytes 4-5:  {payload[4:6].hex().upper()}  → Length = {length} bytes")
    
    # Unit ID
    unit_id = payload[6]
    print(f"  Byte 6:     {payload[6]:02X}  → Unit ID = {unit_id}")
    
    print("\n📋 PDU (Protocol Data Unit)")
    print("-"*70)
    
    # Function Code
    func_code = payload[7]
    functions = {
        0x03: "Read Holding Registers",
        0x04: "Read Input Registers",
        0x06: "Write Single Register"
    }
    func_name = functions.get(func_code, f"Unknown (0x{func_code:02X})")
    print(f"  Byte 7:     {payload[7]:02X}  → Function Code = {func_code} ({func_name})")
    
    # Resto de datos
    if len(payload) > 8:
        print(f"\n  Datos restantes ({len(payload) - 8} bytes):")
        
        # Si es una petición de lectura
        if func_code in [0x03, 0x04] and len(payload) == 12:
            start_addr = struct.unpack('>H', payload[8:10])[0]
            count = struct.unpack('>H', payload[10:12])[0]
            print(f"    Bytes 8-9:   {payload[8:10].hex().upper()}  → Starting Address = {start_addr}")
            print(f"    Bytes 10-11: {payload[10:12].hex().upper()}  → Quantity = {count} registros")
        
        # Si es una respuesta de lectura
        elif func_code in [0x03, 0x04] and len(payload) > 9:
            byte_count = payload[8]
            print(f"    Byte 8:      {payload[8]:02X}  → Byte Count = {byte_count}")
            print(f"    Bytes 9-{8+byte_count}:  {payload[9:9+byte_count].hex().upper()}  → Register Values")
            
            # Decodificar valores
            for i in range(0, byte_count, 2):
                reg_val = struct.unpack('>H', payload[9+i:11+i])[0]
                print(f"      Registro {i//2}: {reg_val}")
        
        # Si es escritura de un registro
        elif func_code == 0x06 and len(payload) == 12:
            reg_addr = struct.unpack('>H', payload[8:10])[0]
            reg_val = struct.unpack('>H', payload[10:12])[0]
            print(f"    Bytes 8-9:   {payload[8:10].hex().upper()}  → Register Address = {reg_addr}")
            print(f"    Bytes 10-11: {payload[10:12].hex().upper()}  → Register Value = {reg_val}")

# Ejemplo: Analizar el primer paquete capturado
if packets_list:
    first_pkt = packets_list[0]
    if Raw in first_pkt:
        payload_hex = bytes(first_pkt[Raw].load).hex()
        analyze_modbus_frame(payload_hex)
else:
    print("⚠️ No hay paquetes para analizar")

---

## 📊 Paso 11: Visualización de Tramas con Scapy

Scapy tiene herramientas integradas para visualizar la estructura de los paquetes.